In [1]:
library(tidyverse)

# Custom package
library(rutils)

-- Attaching packages ---------------------------------------------------------------------- tidyverse 1.3.0 --

v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.0.6     v dplyr   1.0.4
v tidyr   1.1.2     v stringr 1.4.0
v readr   1.4.0     v forcats 0.5.1

-- Conflicts ------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()



In [2]:
dirs <- rutils::get_dev_directories(dev_paths_file = "../dev_paths.txt")
projects <- c("TCGA-CESC", "TCGA-UCS", "TCGA-UCEC", "TCGA-OV")
unified_dsets <- c("unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data")
matrisome_list <- paste0(dirs$data_dir, "/matrisome/matrisome_hs_masterlist.tsv")

In [3]:
dset_idx <- 3
save_lists <- TRUE

In [4]:
p_thresh <- 0.05
lfc_thresh <- log2(2)
# hub_con_thresh <- 0
q_deg_thresh <- 0.05
q_anova_thresh <- 0.05
q_me_thresh <- 0.05
cph_coeff_thresh <- 0.0
q_pbc_thresh <- 0.05
q_cts_thresh <- 0.05
q_univ_surv_thresh <- 0.05
p_mm_thresh <- 0.05

# Data

In [5]:
matrisome_df <- rutils::load_matrisome_df(matrisome_list) %>%
    dplyr::select(gene_symbol, division, category)
norm_counts_df <- read_tsv(paste0(dirs$data_dir, "/", unified_dsets[dset_idx], "/", "norm_counts.tsv"))
m_norm_counts_df <- norm_counts_df %>%
    dplyr::filter(geneID %in% matrisome_df$gene_symbol)

deseq_results_df <- read_tsv(paste0(dirs$analysis_dir, "/deg/", unified_dsets[dset_idx], "_DESeq_results.tsv")) %>%
    rename(base_mean = baseMean, lfc = log2FoldChange)

en_cph_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_en_cph_results.tsv"))
cts_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_survival_cts_results.tsv"))
univ_survival_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_univ_survival_results.tsv"))

en_multinom_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_en_multinom_results.tsv"))
pbc_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_pbc_results.tsv"))
anova_df <- read_tsv(paste0(dirs$analysis_dir, "/feature_selection/", unified_dsets[dset_idx], "_welch_anova_results.tsv"))

network_mm_gs_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_gene_mm_gs.tsv"))
network_me_sig_df <- read_tsv(paste0(dirs$analysis_dir, "/network/", unified_dsets[dset_idx], "_eigengene_traits.tsv"))
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_data.RData"))
lnames <- load(file = paste0(dirs$data_dir, "/saved_network_objects/", unified_dsets[dset_idx], "_tumor_network.RData"))


-- Column specification ---------------------------------------------------------------------------------------
cols(
  Division = col_character(),
  Category = col_character(),
  `Gene Symbol` = col_character(),
  `Gene Name` = col_character(),
  Synonyms = col_character(),
  HGNC_IDs = col_double(),
  `HGNC_IDs Links` = col_double(),
  UniProt_IDs = col_character(),
  Refseq_IDs = col_character(),
  Orthology = col_character(),
  Notes = col_character()
)



-- Column specification ---------------------------------------------------------------------------------------
cols(
  .default = col_double(),
  geneID = col_character()
)
i Use `spec()` for the full column specifications.



-- Column specification ---------------------------------------------------------------------------------------
cols(
  geneID = col_character(),
  baseMean = col_double(),
  log2FoldChange = col_double(),
  lfcSE = col_double(),
  stat = col_double(),
  pvalue = col_double(),
  padj = col_double(),
  qva

# DGE analysis

## DEG

In [6]:
deg_meta_ls <- deg_meta(deseq_results_df, lfc_thresh, q_deg_thresh, nrow(norm_counts_df))
deg_meta_ls[1:4]

$n_deg
[1] 8229

$deg_prop
[1] 0.406531

$n_up
[1] 4862

$n_down
[1] 3367

## DEMG

In [7]:
m_deseq_results_df <- deseq_results_df %>%
    dplyr::filter(geneID %in% matrisome_df$gene_symbol)
demg_meta_ls <- deg_meta(m_deseq_results_df, lfc_thresh, q_deg_thresh, nrow(m_norm_counts_df))
demg_meta_ls[1:4]

$n_deg
[1] 618

$deg_prop
[1] 0.6130952

$n_up
[1] 317

$n_down
[1] 301

In [8]:
matrisome_df %>%
    filter(gene_symbol %in% norm_counts_df$geneID) %>%
    rename(geneID = gene_symbol) %>%
    left_join(deseq_results_df, by = c("geneID")) %>%
    mutate(de = abs(lfc) > lfc_thresh & qval < q_deg_thresh) %>%
    group_by(category) %>%
    summarize(n_cat = n(), pct_de = round(sum(de, na.rm = TRUE) / n_cat * 100))
# deseq_results_df %>%
#     inner_join(matrisome_df, by = c("geneID" = "gene_symbol")) %>%
#     select(geneID, category, lfc, qval) %>%
#     mutate(de = )

,category,n_cat,pct_de
,<chr>,<int>,<dbl>
1,Collagens,43,67
2,ECM Glycoproteins,192,63
3,ECM Regulators,233,61
4,ECM-affiliated Proteins,168,59
5,Proteoglycans,35,69
6,Secreted Factors,337,60


# Uni/multivariate analysis

## FIGO

### LASSO multinomial regression

In [9]:
en_multinom_meta_ls <- list(n_var = nrow(en_multinom_df), genes = en_multinom_df$geneID)
en_multinom_meta_ls[1]

$n_var
[1] 20

### Welch ANOVA

In [10]:
# anova_meta_ls <- simple_test_meta(anova_df, q_anova_thresh)
anova_meta_df <- anova_df %>%
    filter(qval < q_anova_thresh)
anova_meta_ls <- list(n_sig = nrow(anova_meta_df), genes = anova_meta_df$geneID)
anova_meta_ls[1]

$n_sig
[1] 17

### Point-biserial correlation with FIGO

In [11]:
pbc_meta_df <- pbc_df %>%
    condense_figo(include_pvals = TRUE) %>%
    dplyr::filter(figo_min_qval < q_pbc_thresh)
pbc_meta_ls <- list(n_sig = nrow(pbc_meta_df), genes = pbc_meta_df$geneID)
pbc_meta_ls[1]

$n_sig
[1] 3

In [12]:
length(intersect(en_multinom_meta_ls$genes, anova_meta_ls$genes))
length(intersect(en_multinom_meta_ls$genes, pbc_meta_ls$genes))
length(intersect(anova_meta_ls$genes, pbc_meta_ls$genes))

[1] 6

[1] 1

[1] 0

## Survival

### LASSO Cox PH

In [13]:
en_cph_meta_ls <- list(n_var = nrow(en_cph_df), genes = en_cph_df$geneID)
en_cph_meta_ls[1]

$n_var
[1] 48

### Univariate KM/Cox PH

In [14]:
univ_survival_meta_df <- univ_survival_df %>%
    rowwise() %>%
    mutate(min_qval = min(km_qval, cph_qval)) %>%
    as_tibble() %>%
    filter(min_qval < q_univ_surv_thresh)

univ_survival_meta_ls <- list(n_sig = nrow(univ_survival_meta_df), genes = univ_survival_meta_df$geneID)
univ_survival_meta_ls[1]

$n_sig
[1] 3

### Censored time screen

In [15]:
cts_meta_df <- cts_df %>%
    dplyr::filter(vital_qval < q_cts_thresh)
cts_meta_ls <- list(n_sig = nrow(cts_meta_df), genes = cts_meta_df$geneID)
cts_meta_ls[1]

$n_sig
[1] 2

In [16]:
univ_cph_list <- univ_survival_df %>%
    dplyr::filter(cph_qval < q_univ_surv_thresh) %>%
    dplyr::pull(geneID)

length(intersect(cts_meta_ls$genes, univ_cph_list)) / length(union(cts_meta_ls$genes, univ_cph_list))

[1] 0

In [17]:
length(intersect(en_cph_meta_ls$genes, cts_meta_ls$genes))
length(intersect(en_cph_meta_ls$genes, univ_survival_meta_ls$genes))
length(intersect(univ_survival_meta_ls$genes, cts_meta_ls$genes))

[1] 2

[1] 2

[1] 1

# WGCNA

In [18]:
# wgcna_meta_ls <- wgcna_meta(network_me_sig_df, network_mm_gs_df, q_me_thresh, p_mm_thresh, hub_df$geneID)
wgcna_meta_ls <- wgcna_meta(network_me_sig_df, network_mm_gs_df, q_me_thresh, p_mm_thresh, colnames(data_expr))
wgcna_meta_ls[1:2]

$n_sig_modules
[1] 4

$n_sig_genes
[1] 263

In [19]:
wgcna_meta_ls$modules

[1] "green"  "pink"   "salmon" "black"

In [20]:
condensed_network_me_sig_df <- network_me_sig_df %>%
    condense_figo(include_pvals = TRUE) %>%
    dplyr::rename_if(!startsWith(colnames(.), "module"), ~ gsub("^", "me_", .))
condensed_network_me_sig_df %>%
    filter(me_figo_min_qval < q_me_thresh)

module,me_vital_hr,me_vital_dev_cor,me_vital_pval,me_vital_qval,me_figo_min_pval,me_figo_min_qval,me_figo_max_cor
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
green,0.3160419,-0.05760724,0.6402321,0.7891011,0.040950655,0.03632939,0.1278441
black,14.7645533,0.12032337,0.2357867,0.7891011,0.006693521,0.02574670,0.2268016
salmon,0.3447127,-0.03879551,0.6763724,0.7891011,0.021732566,0.03065260,0.1520179
pink,13.5108903,0.12084611,0.1840251,0.7891011,0.025913813,0.03065260,0.1889353


In [21]:
network_mm_gs_df %>%
    select(geneID, module, mm_cor, mm_pval) %>%
    inner_join(condensed_network_me_sig_df, by = "module") %>%
    filter(mm_pval < p_mm_thresh, me_figo_min_qval < q_me_thresh) %>%
    group_by(module) %>%
    summarize(n = n())
    

,module,n
,<chr>,<int>
1,black,122
2,green,69
3,pink,44
4,salmon,28


## TOM meta

In [22]:
colnames(tom) <- colnames(data_expr)
rownames(tom) <- colnames(data_expr)
rowSums(tom) %>%
    quantile() %>%
    round(digits = 1)

0%  25%  50%  75% 100% 
 1.3  2.7  3.6  5.5 27.5

# Create Lists

In [23]:
deg_list <- deg_meta_ls$genes
demg_list <- demg_meta_ls$genes
figo_umsmg_list <- en_multinom_meta_ls$genes %>%
    union(pbc_meta_ls$genes) %>%
    union(anova_meta_ls$genes)
survival_umsmg_list <- en_cph_meta_ls$genes %>%
    union(cts_meta_ls$genes) %>%
    union(univ_survival_meta_ls$genes)
# all_umsmg_list <- figo_umsmg_list %>%
#     intersect(survival_umsmg_list)
figo_nsmg_list <- wgcna_meta_ls$genes
# figo_umsmg_demg_list <- figo_umsmg_list %>%
#     intersect(demg_list)
# survival_umsmg_demg_list <- survival_umsmg_list %>%
#     intersect(demg_list)
# figo_umsmg_nsmg_demg_list <- figo_umsmg_list %>%
#     intersect(figo_nsmg_list) %>%
#     intersect(demg_list)
figo_list <- figo_umsmg_list %>%
    union(figo_nsmg_list) %>%
    intersect(demg_list)
survival_list <- survival_umsmg_list %>%
    intersect(demg_list)
full_overlap_list <- figo_list %>%
    intersect(survival_list)

In [24]:
meta_counts <- tibble(
    n_deg = length(deg_list),
    deg_pct_de = deg_meta_ls$deg_prop,
    deg_up = deg_meta_ls$n_up,
    deg_down = deg_meta_ls$n_down,
    n_demg = length(demg_list),
    demg_pct_de = demg_meta_ls$deg_prop,
    demg_up = demg_meta_ls$n_up,
    demg_down = demg_meta_ls$n_down,
    n_lasso_multinom = en_multinom_meta_ls$n_var,
    n_anova = anova_meta_ls$n_sig,
    n_pbc = pbc_meta_ls$n_sig,
    n_lasso_cox = en_cph_meta_ls$n_var,
    n_uv_cph_km = univ_survival_meta_ls$n_sig,
    n_cts = cts_meta_ls$n_sig,
    n_modules = wgcna_meta_ls$n_sig_modules,
    n_module_genes = wgcna_meta_ls$n_sig_genes,
    n_figo_umsmg = length(figo_umsmg_list),
    n_figo_nsmg = length(figo_nsmg_list),
    n_surv_umsmg = length(survival_umsmg_list),
    n_figo_umsmg_and_demg = length(figo_umsmg_list %>% intersect(demg_list)),
    n_figo_nsmg_and_demg = length(figo_nsmg_list %>% intersect(demg_list)),
    n_figo_umsmg_or_nsmg = union(figo_umsmg_list, figo_nsmg_list) %>% length(),
    n_figo_tot = length(figo_list),
    n_surv_tot = length(survival_list),
    n_full_overlap = length(full_overlap_list)
) %>%
    t()
colnames(meta_counts) <- c("val")
meta_counts_df <- meta_counts %>%
    as_tibble(rownames = "metric") %>%
    mutate(val = round(val, 2))
meta_counts_df

# length(deg_list)
# length(demg_list)
# length(figo_umsmg_list)
# length(figo_nsmg_list)
# length(survival_umsmg_list)
# length(all_umsmg_list)
# length(figo_umsmg_demg_list)
# length(survival_umsmg_demg_list)
# length(figo_umsmg_nsmg_demg_list)

metric,val
<chr>,<dbl>
n_deg,8229.00
deg_pct_de,0.41
deg_up,4862.00
deg_down,3367.00
n_demg,618.00
demg_pct_de,0.61
demg_up,317.00
demg_down,301.00
n_lasso_multinom,20.00


# Save lists

In [25]:
if (save_lists) {
    write_lines(deg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_deg_list.txt"))
    write_lines(demg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_demg_list.txt"))
    write_lines(figo_umsmg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_figo_umsmg_list.txt"))
    write_lines(survival_umsmg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_survival_umsmg_list.txt"))
    write_lines(figo_nsmg_list, paste0(dirs$analysis_dir, "/gene_lists/", unified_dsets[dset_idx], "_figo_nsmg_list.txt"))
    meta_counts_df %>% write_tsv(paste0(dirs$analysis_dir, "/meta/", unified_dsets[dset_idx], "_meta_counts.tsv"))
}